In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import mlflow
import mlflow.keras
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, cohen_kappa_score
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, BatchNormalizationV2
from keras.preprocessing.image import ImageDataGenerator

In [2]:
def datagen():
    train_datagen = ImageDataGenerator(
          rescale=1./255,
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)
    return train_datagen, test_datagen

In [3]:
def train_test_set(train_datagen, test_datagen, train_path, test_path):
    training_set = train_datagen.flow_from_directory(
          train_path,
          target_size=(64, 64),
          batch_size=32,
          class_mode='categorical')
    test_set = test_datagen.flow_from_directory(
          test_path,
          target_size=(64, 64),
          batch_size=32,
          class_mode='categorical',
          shuffle=False)  # Ensure the order of the test set remains the same
    return training_set, test_set

In [4]:
def model(weight_decay):
    cnn = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3],
                               kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                               kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
        tf.keras.layers.BatchNormalization(),  # Add BatchNormalization here
        tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                               kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                               kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
        tf.keras.layers.BatchNormalization(),  # Add BatchNormalization here
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=128, activation='relu',
                              kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    return cnn

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [6]:
def main():
    print("---------------------------Starting---------------------------")
    weight_decay = 1e-4  # Weight decay factor
    learning_rate = 1e-5  # Custom learning rate

    cnn = model(weight_decay)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    cnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    print("defined model, optimizer, and compilation done")
    print("------------defined model, optimizer, and compilation done----------------------")
    train_path = 'dataset\Training'
    test_path = 'dataset/Testing'
    train_datagen, test_datagen = datagen()

    if not os.path.exists(train_path):
        raise FileNotFoundError(f"Training directory not found: {train_path}")
    if not os.path.exists(test_path):
        raise FileNotFoundError(f"Test directory not found: {test_path}")
    
    training_set, test_set = train_test_set(train_datagen, test_datagen, train_path, test_path)
    print("----------------------------training begin---------------------------")

    history = cnn.fit(x=training_set, validation_data=test_set, epochs=10, callbacks=[early_stopping])

    # Get predictions
    y_pred = np.argmax(cnn.predict(test_set), axis=1)
    y_true = test_set.classes  # Directly use the classes attribute
    print("----------------------------training end---------------------------")

if __name__ == '__main__':
    main()

---------------------------Starting---------------------------
defined model, optimizer, and compilation done
------------defined model, optimizer, and compilation done----------------------
Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.
----------------------------training begin---------------------------
Epoch 1/10
90/90 [==============================] - 94s 1s/step - loss: 1.4066 - accuracy: 0.4056 - val_loss: 1.4606 - val_accuracy: 0.1878
Epoch 2/10
90/90 [==============================] - 114s 1s/step - loss: 1.1763 - accuracy: 0.5028 - val_loss: 1.6107 - val_accuracy: 0.1878
Epoch 3/10
90/90 [==============================] - 101s 1s/step - loss: 1.0621 - accuracy: 0.5767 - val_loss: 1.7496 - val_accuracy: 0.1878
Epoch 4/10
90/90 [==============================] - 106s 1s/step - loss: 1.0038 - accuracy: 0.5993 - val_loss: 1.7464 - val_accuracy: 0.2259
Epoch 5/10
90/90 [==============================] - 133s 1s/step - loss: 0.9702 - accuracy: 0